# Switching backends

```{note}
See also {doc}`sympy`.
```

In [ ]:
import logging

import expertsystem as es
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sy
from expertsystem.amplitude.dynamics import (
    relativistic_breit_wigner,
    relativistic_breit_wigner_with_form_factor,
)
from IPython.display import display
from tensorwaves.data.generate import generate_data, generate_phsp
from tensorwaves.estimator import SympyUnbinnedNLL
from tensorwaves.optimizer.minuit import Minuit2
from tensorwaves.physics.amplitude import Intensity, SympyModel
from tensorwaves.physics.helicity_formalism.kinematics import (
    HelicityKinematics,
    ParticleReactionKinematicsInfo,
    SubSystem,
)

logging.getLogger().setLevel(logging.INFO)

In [ ]:
result = es.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=[
        "f(0)(500)",
        "f(0)(980)",
    ],
    formalism_type="helicity",
    topology_building="isobar",
    allowed_interaction_types=["EM", "strong"],
    number_of_threads=1,
)
model_info = es.amplitude.generate_sympy(result)

In [ ]:
dynamics_choices = {
    "D[f_{0}(500) \\to \\pi^{0} \\pi^{0}]": relativistic_breit_wigner_with_form_factor(
        sy.Symbol("m_3+4"),
        sy.Symbol("m_f_(0)(500)"),
        sy.Symbol(R"\Gamma_f_(0)(500)"),
        m_a=0.13497,
        m_b=0.13497,
        angular_momentum=0,
        meson_radius=sy.Symbol("meson_radius_f_(0)(500)"),
    ),
    "D[f_{0}(980) \\to \\pi^{0} \\pi^{0}]": relativistic_breit_wigner_with_form_factor(
        sy.Symbol("m_3+4"),
        sy.Symbol("m_f_(0)(980)"),
        sy.Symbol("\Gamma_f_(0)(980)"),
        m_a=0.13497,
        m_b=0.13497,
        angular_momentum=0,
        meson_radius=sy.Symbol("meson_radius_f_(0)(980)"),
    ),
}

model_info.expression.dynamics.update(
    {
        k: dynamics_choices[k.name]
        for k in model_info.expression.dynamics
        if k.name in dynamics_choices
    }
)

In [ ]:
new_params = es.amplitude.sympy.SuggestedParameterValues()
subs_dict = {}
for par, val in model_info.parameters.items():
    real = sy.Symbol(f"real_{par.name}", real=True)
    imag = sy.Symbol(f"imag_{par.name}", real=True)
    new_params[real] = 1.0
    new_params[imag] = 0.0
    subs_dict[par] = real + sy.I * imag

model_info.expression.amplitudes = {
    k: v.subs(subs_dict) for k, v in model_info.expression.amplitudes.items()
}
model_info.parameters = new_params

In [ ]:
model_info.parameters["m_f_(0)(500)"] = 0.5
model_info.parameters[R"\Gamma_f_(0)(500)"] = 0.2
model_info.parameters["m_f_(0)(980)"] = 0.980
model_info.parameters["\Gamma_f_(0)(980)"] = 0.03
model_info.parameters["meson_radius_f_(0)(500)"] = 1
model_info.parameters["meson_radius_f_(0)(980)"] = 1

In [ ]:
sy_model = SympyModel(
    model_info.expression.full_expression,
    parameters={k: v.value for k, v in model_info.parameters.items()},
    variables={},
)

In [ ]:
def perform_fit(backend: str, analytic_gradient: bool = False) -> None:
    intensity = Intensity(sy_model, backend)
    kinematics = HelicityKinematics(
        ParticleReactionKinematicsInfo(
            initial_state_names=[
                x.name for x in model_info.kinematics.initial_state.values()
            ],
            final_state_names=[
                x.name for x in model_info.kinematics.final_state.values()
            ],
            particles=model_info.particles,
            fs_id_event_pos_mapping=dict(
                {
                    k: i
                    for i, k in enumerate(
                        model_info.kinematics.final_state.keys()
                    )
                }
            ),
        )
    )
    kinematics.register_subsystem(SubSystem([[3, 4], [2]], [], []))
    kinematics.register_subsystem(SubSystem([[3], [4]], [2], []))

    data_sample = generate_data(10_000, kinematics, intensity)
    data_set = kinematics.convert(data_sample)
    data_frame = pd.DataFrame(data_set)

    phsp_sample = generate_phsp(100_000, kinematics)
    phsp_set = kinematics.convert(phsp_sample)

    estimator = SympyUnbinnedNLL(sy_model, data_set, phsp_set, backend=backend)
    starting_values = {
        "m_f_(0)(500)": 0.4,
        R"\Gamma_f_(0)(980)": 0.1,
    }
    intensity.update_parameters(starting_values)
    minuit2 = Minuit2(use_analytic_gradient=analytic_gradient)
    result = minuit2.optimize(estimator, starting_values)
    display(result)
    intensity.update_parameters(result["parameter_values"])
    fitted_data_sample = generate_data(10000, kinematics, intensity)
    fitted_data_set = kinematics.convert(fitted_data_sample)
    fitted_data_frame = pd.DataFrame(fitted_data_set)
    data_frame["m_3+4"].hist(bins=100, alpha=0.5)
    fitted_data_frame["m_3+4"].hist(bins=100, histtype="step", color="red")
    plt.show()

In [ ]:
%time perform_fit("jax", analytic_gradient=True)

In [ ]:
%time perform_fit("jax", analytic_gradient=False)

In [ ]:
%time perform_fit("numpy")